# 17. Recording and playing back data
http://wiki.ros.org/ROS/Tutorials/Recording%20and%20playing%20back%20data

## setup environment

In [1]:
import os
import sys 

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level'

In [2]:
# get ROS related environment variables
envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  
del ROS_ENVs['PWD']

In [3]:
# define workspace path
ws_dir = 'catkin_ws'
ws_dir_abs = os.path.join(home_dir, ws_dir)
src_dir_abs = os.path.join(ws_dir_abs, 'src') 

# append environment paths
def gen_paths(ws_dir_abs, src_dir_abs):
    paths = {}
    paths['ROS_PACKAGE_PATH'] = src_dir_abs
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['ROSLISP_PACKAGE_DIRECTORIES'] = os.path.join(ws_dir_abs + 'devel/share/common-lisp')
    paths['PKG_CONFIG_PATH'] = os.path.join(ws_dir_abs + 'devel/lib/pkgconfig')
    paths['CMAKE_PREFIX_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel')
    return paths

def append_paths(ROS_ENVs, paths): 
    for path in ROS_ENVs.keys():
        os.environ[path] = ROS_ENVs[path]
        if path in paths.keys():
            os.environ[path] = '{}:{}'.format(paths[path], os.environ[path])

append_paths(ROS_ENVs, 
             gen_paths(ws_dir_abs, src_dir_abs))

In [5]:
# define package name and path
package_name = 'beginner_tutorials'

package_dir_abs = !rospack find {package_name}
package_dir_abs = package_dir_abs[0]
package_dir_abs

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials'

## Running the roscore
Make sure that a roscore is up and running:
### roscore
Let's start by making sure that we have roscore running:  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run roscore:
```
$ roscore
```

### turtlesim
For this tutorial we will also use turtlesim.  
open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtlesim_node
```

### turtle keyboard teleoperation
We'll also need something to drive the turtle around with.  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtle_teleop_key
```

Now you can use the arrow keys of the keyboard to drive the turtle around. If you can not drive the turtle, **select the terminal window of the turtle_teleop_key** to make sure that the keys that you type are recorded.

### Recording all published topics
First lets examine the full list of topics that are currently being published in the running system. To do this, open a new terminal and execute the command: 

In [6]:
!rostopic list -v


Published topics:
 * /turtle1/color_sensor [turtlesim/Color] 1 publisher
 * /turtle1/cmd_vel [geometry_msgs/Twist] 1 publisher
 * /rosout [rosgraph_msgs/Log] 3 publishers
 * /rosout_agg [rosgraph_msgs/Log] 1 publisher
 * /turtle1/pose [turtlesim/Pose] 1 publisher

Subscribed topics:
 * /chatter [std_msgs/String] 1 subscriber
 * /turtle1/cmd_vel [geometry_msgs/Twist] 1 subscriber
 * /rosout [rosgraph_msgs/Log] 1 subscriber



The list of published topics are the only message types that could potentially be recorded in the data log file, as only published messages are recorded.  

- The topic /turtle1/cmd_vel is the command message published by teleop_turtle that is taken as input by the turtlesim process.   
- The messages /turtle1/color_sensor and /turtle1/pose are output messages published by turtlesim.

We now will record the published data. Open a new terminal window. In this window run the following commands:

In [7]:
cd {ws_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws


In [8]:
mkdir -p bagfiles

In [9]:
cd bagfiles

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/bagfiles


In [131]:
# !rosbag record -h

In [149]:
# !rostopic list

In [182]:
!rosbag record -O all -e "/turtle1/([A-Za-z0-9_/])+"
# !rosbag record -O subset /turtle1/pose

[ WARN] [1520755004.902393157]: --max-splits is ignored without --split
[ INFO] [1520755004.918800527]: Subscribing to /turtle1/color_sensor
[ INFO] [1520755004.923339557]: Recording to all.bag.
[ INFO] [1520755004.948686652]: Subscribing to /turtle1/cmd_vel
[ INFO] [1520755004.961068590]: Subscribing to /turtle1/pose
^C


Here we are just making a temporary directory to record data and then running rosbag record with the **option -a**, indicating that ***all*** published topics should be accumulated in a bag file.

Move back to the terminal window with turtle_teleop and move the turtle around for 10 or so seconds.

In the window running rosbag record exit with a Ctrl-C. Now examine the contents of the directory ~/bagfiles. You should see a file with a name that begins with the year, date, and time and the suffix .bag. This is the bag file that contains all topics published by any node in the time that rosbag record was running.

## Examining and playing the bag file
Now that we've recorded a bag file using rosbag record we can examine it and play it back using the commands rosbag info and rosbag play. First we are going to see what's recorded in the bag file. We can do the info command -- this command checks the contents of the bag file without playing it back. Execute the following command from the bagfiles directory:

In [183]:
bag_file = sorted(os.listdir())[-1]
bag_file

'all.bag.active'

In [184]:
!rosbag reindex {bag_file}

 all.bag.active                              100%            345.8 KB 00:00    


In [185]:
!rosbag fix {bag_file} output.bag

Bag migrated successfully.


In [186]:
!rosbag info {bag_file}
# !rosbag info output.bag

path:        all.bag.active
version:     2.0
duration:    0.0s
start:       Jan 01 1970 00:00:00.00 (0.00)
end:         Jan 01 1970 00:00:00.00 (0.00)
size:        345.8 KB
messages:    0
compression: none
types:       turtlesim/Color [353891e354491c51aabe32df673fb446]
topics:      /turtle1/color_sensor   0 msg     : turtlesim/Color


This tells us topic names and types as well as the number (count) of each message topic contained in the bag file. We can see that of the topics being advertised that we saw in the rostopic output, four of the five were actually published over our recording interval. As we ran rosbag record with the -a flag it recorded all messages published by all nodes.

###  replay the bag file
The next step in this tutorial is to replay the bag file to reproduce behavior in the running system. First kill the teleop program that may be still running from the previous section - a Ctrl-C in the terminal where you started turtle_teleop_key. Leave turtlesim running. In a terminal window run the following command in the directory where you took the original bag file:

In [158]:
!rosbag play {bag_file}

[ INFO] [1520754594.811119835]: Opening subset.bag.active

Waiting 0.2 seconds after advertising topics... done.

Hit space to toggle paused, or 's' to step.


In its default mode rosbag play will wait for a certain period (.2 seconds) after advertising each message before it actually begins publishing the contents of the bag file. Waiting for some duration allows any subscriber of a message to be alerted that the message has been advertised and that messages may follow. If rosbag play publishes messages immediately upon advertising, subscribers may not receive the first several published messages. The waiting period can be specified with the -d option.

Eventually the topic /turtle1/cmd_vel will be published and the turtle should start moving in turtlesim in a pattern similar to the one you executed from the teleop program. The duration between running rosbag play and the turtle moving should be approximately equal to the time between the original rosbag record execution and issuing the commands from the keyboard in the beginning part of the tutorial. You can have rosbag play not start at the beginning of the bag file but instead start some duration past the beginning using the -s argument. A final option that may be of interest is the -r option, which allows you to change the rate of publishing by a specified factor. If you execute:

In [45]:
!rosbag play -r 2 {bag_file}

[ INFO] [1520738173.154729517]: Opening 2018-03-11-03-15-11.bag.active

No messages to play on specified topics.  Exiting.


You should see the turtle execute a slightly different trajectory - this is the trajectory that would have resulted had you issued your keyboard commands twice as fast.

## Recording a subset of the data
When running a complicated system, such as the pr2 software suite, there may be hundreds of topics being published, with some topics, like camera image streams, potentially publishing huge amounts of data. In such a system it is often impractical to write log files consisting of all topics to disk in a single bag file. The rosbag record command supports logging only particular topics to a bag file, allowing a user to only record the topics of interest to them.

If any turtlesim nodes are running exit them and relaunch the keyboard teleop launch file:
```
rosrun turtlesim turtlesim_node 
rosrun turtlesim turtle_teleop_key
```
In your bagfiles directory, run the following command:

In [46]:
!rosbag record -O subset /turtle1/cmd_vel /turtle1/pose

[ WARN] [1520738249.812323567]: --max-splits is ignored without --split
[ INFO] [1520738249.820792621]: Subscribing to /turtle1/cmd_vel
[ INFO] [1520738249.827638489]: Subscribing to /turtle1/pose
[ INFO] [1520738249.838206803]: Recording to subset.bag.
^C


In [47]:
bag_file = sorted(os.listdir())[-1]
bag_file

'subset.bag'

In [48]:
!rosbag reindex {bag_file}

 subset.bag                                  100%            155.8 KB 00:00    


In [49]:
!rosbag info {bag_file}

path:        subset.bag
version:     2.0
duration:    28.4s
start:       Mar 11 2018 03:17:30.06 (1520738250.06)
end:         Mar 11 2018 03:17:58.45 (1520738278.45)
size:        157.0 KB
messages:    1923
compression: none [1/1 chunks]
types:       geometry_msgs/Twist [9f195f881246fdfa2798d1d3eebca84a]
             turtlesim/Pose      [863b248d5016ca62ea2e895ae5265cf9]
topics:      /turtle1/cmd_vel    148 msgs    : geometry_msgs/Twist
             /turtle1/pose      1775 msgs    : turtlesim/Pose
